###### 2020-10-29 목요일

In [21]:
# 워닝옵션 제거

import warnings
warnings.filterwarnings(action='ignore')

# 07_패딩(Padding)
   - 자연어 처리를 하다보면 각 문장을 토큰화 하는데, 문장을 단어로 토큰화 할때, 문장마다 서로 길이가 다를 수 있다.
   - 문장마다 토큰의 개수가 다르게 생성된다. 이말은 각 문장마다 가지고 있는 데이터의 수가 다르다(길이가 다르다)는 말과 같다 
   - 기계는 행렬(또는 데이터프레임) 형태로 데이터를 처리하는 것이 편하기 때문에, 단어 토큰화된 토큰들을 행렬 형태로 만들어 주어야 한다.
   - 그러므로 행렬 연산을 수행하기 위해 길이가 짧은 데이터에 임의의 숫자(0)를 넣어 행렬의 모양을 동일하게 만들어 주는 작업을 **패딩**이라고 한다.

### 목차

#### 1. Numpy로 패딩하기

#### 2. 케라스 전처리 도구로 패딩하기

## 1. Numpy로 패딩하기

##### 패딩을 실습해보자
##### 우선 문장을 토큰화 해야한다.

In [2]:
text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

In [3]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [36]:
# 텍스트데이터 문장 토큰화 하기
sent_token = sent_tokenize(text)

# 문장 토큰들을 단어 토큰화 하기
word_tokens = [list(word_tokenize(sentence)) for sentence in sent_token]

# 불용어 가져오기
stop_words = set(stopwords.words('english'))

# 불용어 제거 / 정제
result_tokens = []
for tokens in word_tokens :
    temp_list = []
    for token in tokens :
        token = token.lower()    # 대문자를 소문자로
        if (token not in stop_words) & (len(token) > 2):
            temp_list.append(token)
    result_tokens.append(temp_list)
    
print(result_tokens)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [37]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [38]:
# 모든 단어를 정수 인덱스로 부여하자.
tokenizer = Tokenizer()
tokenizer.fit_on_texts(result_tokens)

In [40]:
print(tokenizer.word_index)  # 단어 인덱스사전
print(tokenizer.word_counts) # 단어 빈도사전

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}
OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [52]:
# 제외하는 단어없이 모든 단어에 인덱스를 부여하였다

encoded = tokenizer.texts_to_sequences(result_tokens)
print(encoded)

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


###### 지금까지 과정이 `chapter06_정수인코딩`과정에서 실습했던 내용들이다.

###### 이제 모두 동일한 길이로 패딩하기위해 단어토큰을 가장 많이 포함하는 문장의 토큰 길이를 제보자

In [53]:
max_len = max(list(map(len, encoded)))
print(max_len)

7


 - 가장 긴 문장의 길이(문장이 가지는 토큰의 개수)는 7이다.
##### 이제 7보다 짧은 단어들은 0을 부여하는 작업을 진행해보자

In [54]:
import numpy as np

In [55]:
for element in encoded:
    while len(element) < max_len:
        element.append(0)

padded_tokens = np.array(encoded)
print(padded_tokens)

[list([1, 5]) list([1, 8, 5]) list([1, 3, 5]) list([9, 2])
 list([2, 4, 3, 2]) list([3, 2]) list([1, 4, 6]) list([1, 4, 6])
 list([1, 4, 2]) list([7, 7, 3, 2, 10, 1, 11]) list([1, 12, 3, 13])]
[[ 1  5  0  0  0  0  0]
 [ 1  8  5  0  0  0  0]
 [ 1  3  5  0  0  0  0]
 [ 9  2  0  0  0  0  0]
 [ 2  4  3  2  0  0  0]
 [ 3  2  0  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  2  0  0  0  0]
 [ 7  7  3  2 10  1 11]
 [ 1 12  3 13  0  0  0]]


 - 길이가 7보다 짧은 문장은 전부 숫자 0이 뒬 붙어서 모든 문장의 길이가 전부 7이되었다.
 - 이제 컴퓨터는 이것들을 하나의 행렬(또는 데이터프레임)형태로 데이터를 처리할 수 있다.
 - 단어 토큰을 0번으로 인덱싱 한다는 것은 의미없는 단어를 뜻하는 것이기도 하다.
     - 왜? **빈칸의 정수인덱스도 0 쓸모없는 단어의 정수인덱스도 0 이기 때문이다**
     
     
##### 이와 같이 데이터에 특정 값을 채워서 데이터의 크기를 조정하는 것을 패딩이라고한다.
##### 특히 숫자 0을 통해 패딩을 진행한다면 `제로패딩(Zero Padding)`이라고 한다.

## 2. 케라스 전처리 도구로 패딩하기
   - 케라스는 패딩을 위한 함수인 `pad_sequences()`를 제공하고 있다.

In [60]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [63]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(result_tokens)

encoded = tokenizer.texts_to_sequences(result_tokens)
print(encoded)

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


##### 이까지는 넘파이를 이용한 정수 인덱싱과정과 동일하다

In [66]:
encoded = pad_sequences(encoded)
print(encoded)

[[ 0  0  0  0  0  1  5]
 [ 0  0  0  0  1  8  5]
 [ 0  0  0  0  1  3  5]
 [ 0  0  0  0  0  9  2]
 [ 0  0  0  2  4  3  2]
 [ 0  0  0  0  0  3  2]
 [ 0  0  0  0  1  4  6]
 [ 0  0  0  0  1  4  6]
 [ 0  0  0  0  1  4  2]
 [ 7  7  3  2 10  1 11]
 [ 0  0  0  1 12  3 13]]


 - 위 결과는 넘파이를 이용한 결과와 다르다
 - 만약 넘파이를 이용한 패딩결과와 같은 값을 얻고 싶다면 `padding = post`라는 옵션을 주어야한다

In [70]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(result_tokens)

encoded = tokenizer.texts_to_sequences(result_tokens)
encoded = pad_sequences(encoded, padding='post')
print(encoded)

[[ 1  5  0  0  0  0  0]
 [ 1  8  5  0  0  0  0]
 [ 1  3  5  0  0  0  0]
 [ 9  2  0  0  0  0  0]
 [ 2  4  3  2  0  0  0]
 [ 3  2  0  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  2  0  0  0  0]
 [ 7  7  3  2 10  1 11]
 [ 1 12  3 13  0  0  0]]


 - 케라스는 의미가 없는 단어나 빈칸인 곳을 0이아니라 마지막 인덱스로 부여할 수있다
 - 예를들어 우리의 경우 단어의 정수 인덱스는 1 ~ 13이다.
 - 그러므로 의미 없는 단어나 빈칸인 곳을 14의 인덱스로 부여할 수 있다
 - 실습해보자

In [73]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(result_tokens)

encoded = tokenizer.texts_to_sequences(result_tokens)
encoded = pad_sequences(encoded, padding='post', value=14)

print(encoded)

[[ 1  5 14 14 14 14 14]
 [ 1  8  5 14 14 14 14]
 [ 1  3  5 14 14 14 14]
 [ 9  2 14 14 14 14 14]
 [ 2  4  3  2 14 14 14]
 [ 3  2 14 14 14 14 14]
 [ 1  4  6 14 14 14 14]
 [ 1  4  6 14 14 14 14]
 [ 1  4  2 14 14 14 14]
 [ 7  7  3  2 10  1 11]
 [ 1 12  3 13 14 14 14]]


### 참고

 - 지금 한 실습은 문장이 가지고 있는 토큰의 개수가 가장 많은 문장을 기준으로 padding을 진행하였다.
 - 하지만 꼭 그렇게 하지 않아도 된다.
 - 극단적인 예를 들어보자
     - 평균적으로 문장이 가지고 있는 토큰의 수가 평균 20개이지만,
     - 하나의 문장이 5,000개의 토큰을 가지고 있다면..
     - 꼭 5,000개를 기준으로 진행하지 않아도 된다.